# Geodesic Dome Generator - Test Notebook

This notebook tests the CadQuery-based geodesic dome generator with wedged struts.

## Test Plan
1. Generate a 2V dome at 100cm radius
2. Create just 10 struts first to verify geometry
3. Check that struts meet at vertices without gaps
4. Generate full dome if tests pass


In [ ]:
import sys
sys.path.insert(0, '../src')

import cadquery as cq
from jupyter_cadquery import show, set_defaults

# Set default viewer options
set_defaults(axes=True, axes0=True, grid=True)


In [ ]:
from chapel2 import (
    generate_dome,
    generate_dome_struts_individually,
    generate_2v_test_dome,
    generate_honeycomb_dome,
    export_dome,
)


## Step 1: Generate Geometry Data

First, let's look at the raw geometry data for a 2V dome.


In [ ]:
# Generate honeycomb dome geometry at 100cm radius
radius_cm = 100.0
frequency = 2
portion = 0.5  # hemisphere

vertices, faces, edges = generate_honeycomb_dome(radius_cm, frequency, portion)

print(f"2V Honeycomb Dome at {radius_cm}cm radius:")
print(f"  Vertices: {len(vertices)}")
print(f"  Faces (windows): {len(faces)}")
print(f"  Edges (struts): {len(edges)}")
print(f"\nFirst 5 edges: {edges[:5]}")


## Step 2: Generate 10 Test Struts

Create just 10 struts to verify the wedged strut geometry works correctly.


In [ ]:
# Test parameters
strut_width = 5.0  # cm (~2 inches)
strut_depth = 5.0  # cm (~2 inches)
max_test_struts = 10

# Generate test dome with only 10 struts
test_dome, info = generate_2v_test_dome(
    radius_cm=100.0,
    strut_width=strut_width,
    strut_depth=strut_depth,
    max_struts=max_test_struts
)

print(f"Generated {len(info['edges'])} test struts")
print(f"Strut dimensions: {strut_width}cm x {strut_depth}cm")


In [ ]:
# Visualize the test struts
show(test_dome)


## Step 3: Inspect Individual Struts

Let's look at individual struts to verify the wedged geometry is correct.


In [ ]:
# Generate struts individually for inspection
strut_list, strut_info = generate_dome_struts_individually(
    radius_cm=100.0,
    frequency=2,
    portion=0.5,
    strut_width=5.0,
    strut_depth=5.0,
    dome_style="honeycomb",
    max_struts=3
)

print(f"Generated {len(strut_list)} individual struts for inspection")


In [ ]:
# Show first strut individually
if strut_list:
    show(strut_list[0])


## Step 4: Generate Full 2V Dome

If the test struts look correct, generate the complete 2V dome.


In [ ]:
# Generate full 2V dome
full_2v_dome, full_info = generate_dome(
    radius_cm=100.0,
    frequency=2,
    portion=0.5,
    strut_width=5.0,
    strut_depth=5.0,
    dome_style="honeycomb"
)

print(f"Full 2V Dome:")
print(f"  Vertices: {full_info['num_vertices']}")
print(f"  Edges (struts): {full_info['num_edges']}")
print(f"  Faces (windows): {full_info['num_faces']}")


In [ ]:
# Visualize full 2V dome
show(full_2v_dome)


## Step 5: Generate 3V Chapel Dome (Full Size)

Generate the actual Chapel of MOOP dome at 8ft radius with 3V frequency.


In [ ]:
from chapel2 import generate_chapel_dome

# Generate the Chapel dome
# 8ft radius, 3V frequency, 2" x 2" struts
chapel_dome, chapel_info = generate_chapel_dome(
    radius_ft=8.0,
    frequency=3,
    strut_width_in=2.0,
    strut_depth_in=2.0
)

print(f"Chapel of MOOP Dome (8ft radius, 3V):")
print(f"  Vertices: {chapel_info['num_vertices']}")
print(f"  Struts: {chapel_info['num_edges']}")
print(f"  Windows: {chapel_info['num_faces']}")
print(f"  Radius: {chapel_info['radius_cm']:.1f} cm ({chapel_info['radius_cm']/30.48:.1f} ft)")
print(f"  Strut size: {chapel_info['strut_width']:.2f}cm x {chapel_info['strut_depth']:.2f}cm")


In [ ]:
# Visualize the Chapel dome
show(chapel_dome)


## Step 6: Export Files

Export the dome to STEP and STL files for CAD and 3D printing.


In [ ]:
# Export the 2V test dome
export_dome(
    full_2v_dome,
    step_path="../output/dome_2v.step",
    stl_path="../output/dome_2v.stl"
)


In [ ]:
# Export the Chapel dome
export_dome(
    chapel_dome,
    step_path="../output/chapel_dome_3v.step",
    stl_path="../output/chapel_dome_3v.stl"
)


## Analysis: Window Sizes and Cut List

Generate assembly information for the Chapel dome.


In [ ]:
from chapel2 import analyze_polygon_sizes, generate_assembly_guide, format_assembly_guide_text

# Analyze window sizes
vertices = chapel_info['vertices']
faces = chapel_info['faces']
edges = chapel_info['edges']

sizes = analyze_polygon_sizes(vertices, faces)

if 'hexagons' in sizes:
    h = sizes['hexagons']
    print(f"Hexagons: {h['count']} windows")
    print(f"  Size range: {h['min']:.1f} - {h['max']:.1f} cm")
    print(f"  Average: {h['avg']:.1f} cm ({h['avg']/2.54:.1f} in)")

if 'pentagons' in sizes:
    p = sizes['pentagons']
    print(f"\nPentagons: {p['count']} windows")
    print(f"  Size range: {p['min']:.1f} - {p['max']:.1f} cm")
    print(f"  Average: {p['avg']:.1f} cm ({p['avg']/2.54:.1f} in)")


In [ ]:
# Generate full assembly guide
guide = generate_assembly_guide(
    vertices=vertices,
    edges=edges,
    faces=faces,
    strut_width=chapel_info['strut_width']
)

# Format and print
guide_text = format_assembly_guide_text(
    guide,
    strut_width=chapel_info['strut_width'],
    strut_depth=chapel_info['strut_depth']
)

print(guide_text)


## Done!

The dome generator is working. Check the `output/` directory for the exported files:
- `dome_2v.step` / `dome_2v.stl` - Simple 2V test dome
- `chapel_dome_3v.step` / `chapel_dome_3v.stl` - Full Chapel of MOOP dome
